# A modular GCD algorithm: Overview of algorithm

The individual steps are the same as in K. Weber et al. / Journal of Algorithms 54 (2005) 152–167. Given are two integers $a, b \in \mathbb{Z}^*$. This notebook serves as an overview of the algorithm.

First we define some common functions needed by the algorithm.

In [1]:
def mod(b, m):
    """
    Returns the symmetrical modular representation of b mod n
    
    Parameters
    ----------
    b: integer
        An integer.
    m: integer
        The modulus.
        
    Returns
    -------
    y: integer
        Symmetrical modular representation b mod n.
    """
    y = b % m
    if y > m / 2:
        y -= m

    return y

In [2]:
def pi(x):
    """
    Calculates the number of primes less than or equal to the given real number
    
    Parameters
    ----------
    x: real
        A real number.
        
    Returns
    -------
    pi: integer
        Number of primes.
    """
    P = Primes()
    i = 0
    while P.unrank(i) <= x:
        i += 1

    return i

In [3]:
def getPrimes(n):
    primes = set()
    P = Primes()
    i = max(n, 9)
    while len(primes) < n + 2:
        i += 1
        primes.add(P.unrank(i))

    return primes

In [4]:
def findPrime(P, U, V):
    minimum = None
    cur_p = None
    for p in P:
        m = abs(mod(U / V, p))
        if minimum == None or m < minimum:
            minimum = m
            cur_p = p
    
    return cur_p

In [5]:
def getPrimes_mod(number, bound):
    primes = set()
    P = Primes()
    i = 0
    while P.unrank(i) < bound:
        i += 1

    while len(primes) < number:
        assert(i > 0)
        i -= 1
        primes.add(P.unrank(i))

    return primes

In [6]:
def findPrime_mod(P, u, v):
    minimum = None
    current_p = None
    for p in P:
        if v[p] != 0:
            m = abs(mod(u[p] / v[p], p))
            if minimum == None or m < minimum:
                minimum = m
                current_p = p

    return current_p    

Now we implement the algorithm.

In [7]:
def gcd_int(a, b):
    if a >= b:
        U, V = a, b
    else:
        U, V = b, a

    n = floor(log(U, 2)) + 1
    Q = getPrimes(n)
    while V != 0:
        P = set()
        for q in Q:
            if mod(V, q) != 0:
                P.add(q)

        p = findPrime(P, U, V)
        b = mod(U / V, p)
        Q.remove(p)
        U, V = V, (U - b * V) / p
    
    return abs(U)

In [8]:
def gcd_int_mod(a, b):
    if a >= b:
        U, V = a, b
    else:
        U, V = b, a

    n = floor(log(U, 2)) + 1
    w = 1
    while pi(2^w) - pi(2^(w-1)) < max(ceil(2^(w/2) + n), 9):
        w += 1
    
    Q = getPrimes_mod(ceil(2^(w/2) + n), 2^w)
    u, v = {}, {}
    for q in Q:
        u[q], v[q] = mod(U, q), mod(V, q)

    while any(x != 0 for x in v.values()):
        P = set()
        for q in Q:
            if mod(V, q) != 0:
                P.add(q)

        p = findPrime_mod(P, u, v)
        b = mod(u[p] / v[p], p)
        Q.remove(p)
        del u[p], v[p]
        for q in Q:
            u[q], v[q] = v[q], mod((u[q] - b * v[q]) / p, q)    

    G = 0
    M = 1
    r = u.copy()
    while any(x != 0 for x in r.values()):
        p = Q.pop()
        G += M * r[p]
        M *= p
        for q in Q:
            r[q] = mod((u[q] - G) / M, q)

        del u[p]
        del r[p]

    return abs(G)

Next we test our algorithm on simple inputs.

In [9]:
gcd_int(7 * 5 * 7 * 7 * 3, 7 * 5 * 11 * 5)

35

In [10]:
gcd_int_mod(7 * 5 * 7 * 7 * 3, 7 * 5 * 11 * 5)

35

In [11]:
assert gcd_int(1, 1) == 1
assert gcd_int_mod(1, 1) == 1

In [12]:
assert gcd_int(3, 1) == 1
assert gcd_int_mod(3, 1) == 1

In [13]:
assert gcd_int(1, 3) == 1
assert gcd_int_mod(1, 3) == 1

In [14]:
assert gcd_int(7 * 13 * 19, 11 * 17) == 1
assert gcd_int_mod(7 * 13 * 19, 11 * 17) == 1

In [15]:
def createExample(size):
    P = Primes()
    gcd = 1
    a = 1
    b = 1
    for i in range(size):
        gcd *= P.unrank(i)
        a *= P.unrank(i) * P.unrank(size + 2 * i + 1)
        b *= P.unrank(i) * P.unrank(size + 2 * i)

    return a, b, gcd

In [16]:
a, b, gcd = createExample(3)
assert gcd_int(a, b) == gcd
assert gcd_int_mod(a, b) == gcd

In [17]:
a, b, gcd = createExample(6)
assert gcd_int(a, b) == gcd
assert gcd_int_mod(a, b) == gcd

In [18]:
a, b, gcd = createExample(9)
assert gcd_int(a, b) == gcd
assert gcd_int_mod(a, b) == gcd

Finally, we do some performance tests.

In [19]:
for size in range(10, 130, 10):
    a, b, gcd = createExample(size)
    print 'size ==', size
    print 'gcd_int'
    timeit('gcd_int(a, b)')
    print 'gcd_int_mod'
    timeit('gcd_int_mod(a, b)')
    print

size == 10
gcd_int
25 loops, best of 3: 9.24 ms per loop
gcd_int_mod
25 loops, best of 3: 24.4 ms per loop

size == 20
gcd_int
5 loops, best of 3: 51.2 ms per loop
gcd_int_mod
5 loops, best of 3: 99.7 ms per loop

size == 30
gcd_int
5 loops, best of 3: 119 ms per loop
gcd_int_mod
5 loops, best of 3: 186 ms per loop

size == 40
gcd_int
5 loops, best of 3: 223 ms per loop
gcd_int_mod
5 loops, best of 3: 338 ms per loop

size == 50
gcd_int
5 loops, best of 3: 392 ms per loop
gcd_int_mod
5 loops, best of 3: 545 ms per loop

size == 60
gcd_int
5 loops, best of 3: 630 ms per loop
gcd_int_mod
5 loops, best of 3: 808 ms per loop

size == 70
gcd_int
5 loops, best of 3: 970 ms per loop
gcd_int_mod
5 loops, best of 3: 1.14 s per loop

size == 80
gcd_int
5 loops, best of 3: 1.37 s per loop
gcd_int_mod
5 loops, best of 3: 1.53 s per loop

size == 90
gcd_int
5 loops, best of 3: 1.89 s per loop
gcd_int_mod
5 loops, best of 3: 1.98 s per loop

size == 100
gcd_int
5 loops, best of 3: 2.46 s per loop
gc